In [18]:
import os
import pandas as pd
from tqdm import tqdm
import math

In [19]:
OUT_PATH = 'nodes'
SCHEMAS = 'node_template/submission_demographic_template.tsv'

In [20]:
#read the diagnosis node header
demographic_node = pd.read_csv(SCHEMAS, sep = '\t')
demographic_node.drop(index=demographic_node.index[0], 
        axis=0, 
        inplace=True)
demographic_node_qc = demographic_node.copy()
demographic_node

,*type,project_id,*submitter_id,*cases.submitter_id,age_at_index,cause_of_death_primary,cause_of_death_secondary,cur_employ_stat,days_to_death,death_related_to,education,ethnicity,gender,marital,race,sex,vital_status,year_of_birth,year_of_death


In [21]:
demographics_df = pd.read_csv('csv_datasets/DEMOGRAPHICS.csv')
demographics_df

,usubjid,site,init,age,brthdtc,sex,genderoth,race,ethnic,scdat,...,rct_eosreason,rct_eosreasonoth,rct_eosreason1,rct_eosreason1a,rct_eosreason1adat,rct_eosreason1b,rct_eosreason1bdat,eos_sourcedat,eos_sourcesig,end_of_study_complete
0,1001,1,IJP,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,1,M-H,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,1,K-P,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,1,J-M,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,1,R-S,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,81149,Virginia Commonwealth University,NaN,58.046,1964-08-09,Female,NaN,White,Non-hispanic,2022-08-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1522,81150,Virginia Commonwealth University,NaN,61.504,1961-02-28,Male,NaN,Black or African American,Non-hispanic,2022-08-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1523,81151,Virginia Commonwealth University,NaN,42.186,1980-07-08,Male,NaN,White,Non-hispanic,2022-09-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
1524,81152,Virginia Commonwealth University,NaN,37.961,1984-09-30,Male,NaN,White,Non-hispanic,2022-09-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Read case node data
case_node_pd = pd.read_csv('nodes/case_node.tsv', sep = '\t')
case_node_pd = case_node_pd.iloc[: , 1:]
case_node_pd

,project_id,*submitter_id,*studies.submitter_id,actarm,ah_hosp,ah_hosp_num,aki_status,bari_surgery,cohort,consent_type,...,days_90_aki,days_90_survival,days_to_aki,days_to_consent,days_to_death,index_date,inf_cnst_sign_dt,rct_meld_strata,study_site,vital_status
0,ardac-p1031,11001_obs,obs,,,,Unknown,,Healthy Donor,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Cleveland Clinic,alive
1,ardac-p1031,11002_obs,obs,,,,Unknown,,Healthy Donor,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Cleveland Clinic,alive
2,ardac-p1031,11003_obs,obs,,,,Unknown,,Heavy Drinker with Alcoholic Hepatits,,...,Unknown,alive,NaN,,NaN,Study Enrollment,,,Cleveland Clinic,alive
3,ardac-p1031,11004_obs,obs,,,,Unknown,,Healthy Donor,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Cleveland Clinic,alive
4,ardac-p1031,11006_obs,obs,,,,Unknown,,Heavy Drinker with Alcoholic Hepatits,,...,Unknown,alive,NaN,,100.0,Study Enrollment,,,Cleveland Clinic,dead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,ardac-p1031,81149_obs,obs,,,,Unknown,,Heavy Drinker without Alcoholic Hepatits,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Virginia Commonwealth University,alive
898,ardac-p1031,81150_obs,obs,,,,Unknown,,Heavy Drinker without Alcoholic Hepatits,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Virginia Commonwealth University,alive
899,ardac-p1031,81151_obs,obs,,,,Unknown,,Heavy Drinker with Alcoholic Hepatits,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Virginia Commonwealth University,alive
900,ardac-p1031,81152_obs,obs,,,,Unknown,,Heavy Drinker with Alcoholic Hepatits,,...,Unknown,Unknown,NaN,,NaN,Study Enrollment,,,Virginia Commonwealth University,alive


In [23]:
def row_init(len):
    row = [None]*len
    return row

def set_column_value(df, col_name, row, value):
    index= df.columns.get_loc(col_name)
    row[index] = value
    return row

In [24]:
for index,row in tqdm(demographics_df.iterrows(),total=demographics_df.shape[0],desc="process enrollment"):
    if row.usubjid <10000:
        continue
        
    demographics_type = 'demographic'
    project_id = 'ardac-p1031'
    
    demographics_case_submitter_id = row.usubjid
    for index_case,row_case in case_node_pd.iterrows():
        if str(demographics_case_submitter_id) in str(row_case['*submitter_id']):
            demographics_case_submitter_id = str(row_case['*submitter_id'])
            break
    
    case_row = case_node_pd.loc[case_node_pd['*submitter_id'] == demographics_case_submitter_id]
    
    #print(case_row['*studies.submitter_id'])
    
    
    #if there is a demographic can not find a studies.submitter_id from CASE node, report to QC.
    if case_row['*studies.submitter_id'].empty:
        demographics_submitter_id = None    
        sex = row.sex
        age_at_index = row.age
    
        year_of_birth = str(row.brthdtc).split('-')[0]
        demographics_row_qc = row_init(len(demographic_node.axes[1]))
        demographics_row_qc = set_column_value(demographic_node_qc, '*type', demographics_row_qc, demographics_type)
        demographics_row_qc = set_column_value(demographic_node_qc, 'project_id', demographics_row_qc, project_id)
        demographics_row_qc = set_column_value(demographic_node_qc, '*submitter_id', demographics_row_qc, demographics_submitter_id)
        demographics_row_qc = set_column_value(demographic_node_qc, '*cases.submitter_id', demographics_row_qc, demographics_case_submitter_id)
        demographics_row_qc = set_column_value(demographic_node_qc, 'age_at_index', demographics_row_qc, age_at_index)
        demographics_row_qc = set_column_value(demographic_node_qc, 'sex', demographics_row_qc, sex)
        demographics_row_qc = set_column_value(demographic_node_qc, 'year_of_birth', demographics_row_qc, year_of_birth)
           
        demographic_node_qc.loc[len(demographic_node_qc.index)] = demographics_row_qc
        continue
        
        
    else:
        demographics_submitter_id = str(case_row['*studies.submitter_id'].item()) + '_' + str(demographics_case_submitter_id)
    
    
    sex = row.sex
    gender = row.gender
    race = row.race
    ethnicity = row.ethnic
    
    if row.age == row.age: #if the age != NaN
        age_at_index = math.ceil(row.age)
    else:                  # age = NaN
        age_at_index = row.age
    year_of_birth = str(row.brthdtc).split('-')[0]
    
    
    demographics_row = row_init(len(demographic_node.axes[1]))
    demographics_row = set_column_value(demographic_node, '*type', demographics_row, demographics_type)
    demographics_row = set_column_value(demographic_node, 'project_id', demographics_row, project_id)
    demographics_row = set_column_value(demographic_node, '*submitter_id', demographics_row, demographics_submitter_id)
    demographics_row = set_column_value(demographic_node, '*cases.submitter_id', demographics_row, demographics_case_submitter_id)
    demographics_row = set_column_value(demographic_node, 'age_at_index', demographics_row, age_at_index)
    demographics_row = set_column_value(demographic_node, 'sex', demographics_row, sex)
    demographics_row = set_column_value(demographic_node, 'gender', demographics_row, gender)
    demographics_row = set_column_value(demographic_node, 'race', demographics_row, race)
    demographics_row = set_column_value(demographic_node, 'ethnicity', demographics_row, ethnicity)
    demographics_row = set_column_value(demographic_node, 'year_of_birth', demographics_row, year_of_birth)
    
    demographic_node.loc[len(demographic_node.index)] = demographics_row

demographic_node

process enrollment: 100%|███████████████████| 1526/1526 [00:18<00:00, 80.81it/s]


,*type,project_id,*submitter_id,*cases.submitter_id,age_at_index,cause_of_death_primary,cause_of_death_secondary,cur_employ_stat,days_to_death,death_related_to,education,ethnicity,gender,marital,race,sex,vital_status,year_of_birth,year_of_death
0,demographic,ardac-p1031,obs_11001_obs,11001_obs,58.0,None,None,None,None,None,None,Non-hispanic,Male,None,White,Male,None,1962,None
1,demographic,ardac-p1031,obs_11002_obs,11002_obs,61.0,None,None,None,None,None,None,Non-hispanic,Female,None,Native Hawaiian or Other Pacific Islander,Female,None,1958,None
2,demographic,ardac-p1031,obs_11003_obs,11003_obs,36.0,None,None,None,None,None,None,Non-hispanic,Male,None,White,Male,None,1983,None
3,demographic,ardac-p1031,obs_11004_obs,11004_obs,55.0,None,None,None,None,None,None,Non-hispanic,Male,None,White,Male,None,1964,None
4,demographic,ardac-p1031,obs_11006_obs,11006_obs,50.0,None,None,None,None,None,None,Non-hispanic,Male,None,White,Male,None,1970,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,demographic,ardac-p1031,obs_81149_obs,81149_obs,59.0,None,None,None,None,None,None,Non-hispanic,Female,None,White,Female,None,1964,None
904,demographic,ardac-p1031,obs_81150_obs,81150_obs,62.0,None,None,None,None,None,None,Non-hispanic,Male,None,Black or African American,Male,None,1961,None
905,demographic,ardac-p1031,obs_81151_obs,81151_obs,43.0,None,None,None,None,None,None,Non-hispanic,Male,None,White,Male,None,1980,None
906,demographic,ardac-p1031,obs_81152_obs,81152_obs,38.0,None,None,None,None,None,None,Non-hispanic,Male,None,White,Male,None,1984,None


In [25]:
for index,row in tqdm(demographic_node.iterrows(),total=demographic_node.shape[0],desc="Cleaning Demographic Node"):
    if row.race == 'White':
        demographic_node.at[index,'race'] = 'white'
    if row.race == 'Asian':
        demographic_node.at[index,'race'] = 'asian'
    if row.race == 'Black or African American':
        demographic_node.at[index,'race'] = 'black or african american'
    if row.race == 'Multiple races':
        demographic_node.at[index,'race'] = 'More than one race'
    if row.race == 'Native Hawaiian or Other Pacific Islander':
        demographic_node.at[index,'race'] = 'native hawaiian or other pacific islander'
    if row.race == 'American Indian or Alaska Native':
        demographic_node.at[index,'race'] = 'american indian or alaska native'        
    if row.ethnicity == 'Non-hispanic':
        demographic_node.at[index,'ethnicity'] = 'not hispanic or latino'
    if row.ethnicity == 'Hispanic or Latino':
        demographic_node.at[index,'ethnicity'] = 'hispanic or latino'
    if row.ethnicity == None:
        demographic_node.at[index,'ethnicity'] = 'not reported'
    if row.gender == 'Male':
        demographic_node.at[index,'gender'] = 'male'
    if row.gender == 'Female':
        demographic_node.at[index,'gender'] = 'female'
    if row.gender == None:
        demographic_node.at[index,'gender'] = 'not reported'
    if row.sex == 'Male':
        demographic_node.at[index,'sex'] = 'M'
    if row.sex == 'Female':
        demographic_node.at[index,'sex'] = 'F'
    if row.sex == None:
        demographic_node.at[index,'sex'] = 'U'

Cleaning Demographic Node: 100%|████████████| 908/908 [00:00<00:00, 5210.64it/s]


In [26]:
#output the case node
demographic_node_path = os.path.join(OUT_PATH, 'demographic_node.tsv')
demographic_node.to_csv(demographic_node_path, sep = '\t', index=False)

demographic_node_qc_path = os.path.join(OUT_PATH, 'demographic_node_qc.tsv')
demographic_node_qc.to_csv(demographic_node_qc_path, sep = '\t', index=False)

demographic_node

,*type,project_id,*submitter_id,*cases.submitter_id,age_at_index,cause_of_death_primary,cause_of_death_secondary,cur_employ_stat,days_to_death,death_related_to,education,ethnicity,gender,marital,race,sex,vital_status,year_of_birth,year_of_death
0,demographic,ardac-p1031,obs_11001_obs,11001_obs,58.0,None,None,None,None,None,None,not hispanic or latino,male,None,white,M,None,1962,None
1,demographic,ardac-p1031,obs_11002_obs,11002_obs,61.0,None,None,None,None,None,None,not hispanic or latino,female,None,native hawaiian or other pacific islander,F,None,1958,None
2,demographic,ardac-p1031,obs_11003_obs,11003_obs,36.0,None,None,None,None,None,None,not hispanic or latino,male,None,white,M,None,1983,None
3,demographic,ardac-p1031,obs_11004_obs,11004_obs,55.0,None,None,None,None,None,None,not hispanic or latino,male,None,white,M,None,1964,None
4,demographic,ardac-p1031,obs_11006_obs,11006_obs,50.0,None,None,None,None,None,None,not hispanic or latino,male,None,white,M,None,1970,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,demographic,ardac-p1031,obs_81149_obs,81149_obs,59.0,None,None,None,None,None,None,not hispanic or latino,female,None,white,F,None,1964,None
904,demographic,ardac-p1031,obs_81150_obs,81150_obs,62.0,None,None,None,None,None,None,not hispanic or latino,male,None,black or african american,M,None,1961,None
905,demographic,ardac-p1031,obs_81151_obs,81151_obs,43.0,None,None,None,None,None,None,not hispanic or latino,male,None,white,M,None,1980,None
906,demographic,ardac-p1031,obs_81152_obs,81152_obs,38.0,None,None,None,None,None,None,not hispanic or latino,male,None,white,M,None,1984,None


In [27]:
demographic_node_qc

,*type,project_id,*submitter_id,*cases.submitter_id,age_at_index,cause_of_death_primary,cause_of_death_secondary,cur_employ_stat,days_to_death,death_related_to,education,ethnicity,gender,marital,race,sex,vital_status,year_of_birth,year_of_death
0,demographic,ardac-p1031,None,11005,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
1,demographic,ardac-p1031,None,11016,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
2,demographic,ardac-p1031,None,11037,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
3,demographic,ardac-p1031,None,11072,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
4,demographic,ardac-p1031,None,11126,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
5,demographic,ardac-p1031,None,11148,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
6,demographic,ardac-p1031,None,21017,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
7,demographic,ardac-p1031,None,21027,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
8,demographic,ardac-p1031,None,21030,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
9,demographic,ardac-p1031,None,21033,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,nan,None
